In [190]:
def layer_cost_func(layer):
    cost=[200,1,0,0,0,0.5,3]
    return cost[layer]

class SearchEntry_3d():
    def __init__(self, x, y, z, g_cost, c_cost ,f_cost=0, pre_entry=None):
        self.x = x
        self.y = y
        self.z = z
        # cost move form start entry to this entry
        self.g_cost = g_cost
        self.c_cost = c_cost
        self.f_cost = f_cost
        self.pre_entry = pre_entry

    def getPos(self):
        return (self.x, self.y, self.z)

class Map_3d():
    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth
        self.map = [[[" " for x in range(self.width)] for y in range(self.height)] for z in range(self.depth)]

    def setBlock(self,block_list): #block list 储存已经存在的单元位置信息
        block_list_len=len(block_list)
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            z=block_list[i][2]
            if(x>=0 and y>=0 and x<self.width and y<self.height):
                self.map[z][y][x] = z; 
    def setBlock_test(self,block_list,index): #block list 储存已经存在的单元位置信息
        block_list_len=len(block_list)
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            z=block_list[i][2]
            if(x>=0 and y>=0 and x<self.width and y<self.height):
                self.map[index][y][x] = z; 

    def setBlock_layout(self,block_list,z): #block list 储存已经存在的单元位置信息
        block_list_len=len(block_list)
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            if(x>=0 and y>=0 and x<self.width and y<self.height):
                self.map[z][y][x] = z;                 
    
    def clrBlock(self,block_list): #清除block list
        block_list_len=len(block_list)
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            z=block_list[i][2]
            if(x>=0 and y>=0 and x<self.width and y<self.height):
                self.map[z][y][x] = " ";

    def clrBlock_layout(self,block_list,z): #清除block list
        block_list_len=len(block_list)
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            if(x>=0 and y>=0 and x<self.width and y<self.height):
                self.map[z][y][x] = " ";   
         
    def get_blocked_point(self,block_list,z):#用来检查block_list里的点是否原来就是封死的，返回一个原来是封死的list，防止clrblock时把原来的也清掉了
        block_list_len=len(block_list)
        prev_block=[]
        for i in range(0,block_list_len):
            x=block_list[i][0]
            y=block_list[i][1]
            if(x>=0 and y>=0 and x<self.width and y<self.height):
                if(self.map[z][y][x]==z):
                    prev_block.append([x,y])
        return prev_block

    
    def showMap(self,z):
        print("+" * (3 * self.width + 2))
        len_row=len(self.map[z])
        #print(len_row)
        for i in range(len_row):
            s = '+'
            len_col=len(self.map[z][len_row-i-1])
            for j in range(len_col):
                if(self.map[z][len_row-i-1][j]==0):
                    s +=' '+ "0" + ' '
                else:
                    s +=' '+ str(self.map[z][len_row-i-1][j]) + ' '                   
            s +='+'
            print(s)

        print("+" * (3 * self.width + 2))
        
    def createBlock(self, block_num,z):
        for i in range(block_num):
            x, y = (randint(0, self.width-1), randint(0, self.height-1))
            self.map[z][y][x] = "N"

    def copyMap(self,map_reg,z):
        for x in range(self.width):
            for y in range(self.height):
                map_reg.setValue([x,y],self.map[z][y][x])
    def restoreMap(self,map_reg,z):
        for x in range(self.width):
            for y in range(self.height):
                self.map[z][y][x]=map_reg.getValue([x,y])  

    def get_util(self):
        total=self.width*self.height*self.depth
        used=0.0
        for x in range(self.width):
            for y in range(self.height):
                for z in range(self.depth):
                    if self.map[z][y][x]==z:
                        used+=1.0
        return [total,used,float(used/total)]
def AStarSearch_3d(map, source, dest):
    def getNewPosition(map, locatioin, offset):
        x,y,z = (location.x + offset[0], location.y + offset[1], location.z + offset[2])
        pos=(x,y,z)
        if x < 0 or x >= map.width or y < 0 or y >= map.height or z<0 or z>=map.depth or map.map[z][y][x] == z or  map.map[z][y][x] == "N":
            pos=None  #节点限制在map范围内，并且绕开屏蔽点
            
        else:
            if(location.c_cost != 0):#当前为拐角时不进行z方向移动
                if(offset[2]!=0):
                    pos=None
                
            if(location.pre_entry != None and location.pre_entry.pre_entry != None):#检测从高层走到底层原路范围的情况
                if(location.pre_entry.pre_entry.x==x and location.pre_entry.pre_entry.y==y):
                    pos=None
                if(location.pre_entry.z != location.pre_entry.pre_entry.z):
                    if(abs(location.pre_entry.x-x)==1 and abs(location.pre_entry.y-1)==1):
                        pos=None
                    elif(offset[2]!=0):
                        pos=None
        return pos

    def getPositions(map, location):#这里设置寻路类型，使用曼哈顿或折线距离
        # use four ways or eight ways to move 
        if(location.pre_entry != None and location.pre_entry.pre_entry != None): #非第二个节点
            pre_z=location.pre_entry.z
            corner= abs(location.pre_entry.pre_entry.x-location.x)==1 and abs(location.pre_entry.pre_entry.y-location.y)==1 

            if(location.pre_entry.z != location.pre_entry.pre_entry.z):#如果上一个节点换层，屏蔽上下换层方向/拐角方向
                if(abs(location.pre_entry.x-location.x)==1): #x方向上前进，屏蔽y方向拐角
                    offsets = [(-1,0,0),  (1,0,0)]
                elif(abs(location.pre_entry.y-location.y)==1): #y方向上前进，屏蔽x方向拐角
                    offsets = [(0,1,0),  (0,-1,0)]
                else:#待定
                    offsets = [(0,1,0),  (0,-1,0),(-1,0,0),  (1,0,0)]
            elif(location.z!=location.pre_entry.z):
                if(abs(location.x-location.pre_entry.pre_entry.x)==1):#x方向前进屏蔽y拐角
                    offsets = [(-1,0,0), (1,0,0)]#屏蔽上下节点和y拐角
                elif(abs(location.y-location.pre_entry.pre_entry.y)==1):
                    offsets = [(0,-1,0), (0,1,0)]#屏蔽上下节点和x拐角
            elif(corner):
                if(abs(location.pre_entry.pre_entry.x-location.pre_entry.x)==1):
                    offsets =  [(0,-1,0), (0, 1,0),(0,0,1),(0,0,-1)]
                elif(abs(location.pre_entry.pre_entry.y-location.pre_entry.y)==1):
                    offsets = [(-1,0,0), (1,0,0), (0,0,1),(0,0,-1)]
                else:
                    offsets = [(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)]#如果没有上个节点换层则按照cost来处理
            elif(pre_z==0):
                if(map.map[1][location.pre_entry.y][location.pre_entry.x]==1):
                    if(abs(location.pre_entry.x-location.x)==1):
                        offsets = [(-1,0,0),  (1,0,0)]    
                    else:
                        offsets = [(0,1,0),  (0,-1,0)]        
                else:
                    offsets=[(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)]
            elif(pre_z==map.depth-1):
                if(map.map[map.depth-2][location.pre_entry.y][location.pre_entry.x]==map.depth-2):
                    if(abs(location.pre_entry.x-location.x)==1):
                        offsets = [(-1,0,0),  (1,0,0)]    
                    else:
                        offsets = [(0,1,0),  (0,-1,0)]        
                else:
                    offsets=[(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)]  
            elif( pre_z>0 and pre_z<map.depth-1):
                if(map.map[pre_z+1][location.pre_entry.y][location.pre_entry.x]==pre_z+1):
                    if(abs(location.pre_entry.x-location.x)==1):
                        offsets = [(-1,0,0),  (1,0,0)]    
                    else:
                        offsets = [(0,1,0),  (0,-1,0)]
                elif(map.map[pre_z-1][location.pre_entry.y][location.pre_entry.x]==pre_z-1):
                    if(abs(location.pre_entry.x-location.x)==1):
                        offsets = [(-1,0,0),  (1,0,0)]    
                    else:
                        offsets = [(0,1,0),  (0,-1,0)]            
                else:
                    offsets=[(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)] 
            else:
                 offsets=[(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)] 
        elif(location.pre_entry != None):#非第一个节点
            pre_z=location.pre_entry.z
            if(location.z != location.pre_entry.z):#如果当前位置和上一位置换层
                if(location.pre_entry.pre_entry != None and abs(location.x-location.pre_entry.pre_entry.x)==1):#x方向前进屏蔽y拐角
                    offsets = [(-1,0,0), (1,0,0)]#屏蔽上下节点和y拐角
                elif(location.pre_entry.pre_entry != None and abs(location.y-location.pre_entry.pre_entry.y)==1):
                    offsets = [(0,-1,0), (0,1,0)]#屏蔽上下节点和x拐角
                else:#待定
                    offsets = [(0,1,0),  (0,-1,0),(-1,0,0),  (1,0,0)]            
            elif(pre_z==0):
                if(map.map[1][location.pre_entry.y][location.pre_entry.x]==1):
                    if(abs(location.pre_entry.x-location.x)==1):
                        offsets = [(-1,0,0),  (1,0,0)]    
                    else:
                        offsets = [(0,1,0),  (0,-1,0)]        
                else:
                    offsets=[(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)]
            elif(pre_z==map.depth-1):
                if(map.map[map.depth-2][location.pre_entry.y][location.pre_entry.x]==map.depth-2):
                    if(abs(location.pre_entry.x-location.x)==1):
                        offsets = [(-1,0,0),  (1,0,0)]    
                    else:
                        offsets = [(0,1,0),  (0,-1,0)]        
                else:
                    offsets=[(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)]  
            elif( pre_z>0 and pre_z<map.depth-1):
                if(map.map[pre_z+1][location.pre_entry.y][location.pre_entry.x]==pre_z+1):
                    if(abs(location.pre_entry.x-location.x)==1):
                        offsets = [(-1,0,0),  (1,0,0)]    
                    else:
                        offsets = [(0,1,0),  (0,-1,0)]
                elif(map.map[pre_z-1][location.pre_entry.y][location.pre_entry.x]==pre_z-1):
                    if(abs(location.pre_entry.x-location.x)==1):
                        offsets = [(-1,0,0),  (1,0,0)]    
                    else:
                        offsets = [(0,1,0),  (0,-1,0)]            
                else:
                    offsets=[(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)]
            else:#如果没换层则不处理
                offsets = [(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0),(0,0,1),(0,0,-1)]

                
        else:#第一个拐角去掉换层z方向
            offsets = [(-1,0,0), (0,-1,0), (1,0,0), (0, 1,0)]
        poslist = []
        for offset in offsets:
            pos = getNewPosition(map, location, offset)
            if pos is not None:
                poslist.append(pos)
        return poslist

    # imporve the heuristic distance more precisely in future
    def calHeuristic(pos, dest):
        return abs(dest.x - pos[0]) + abs(dest.y - pos[1])+abs(dest.z - pos[2])
        
    def getMoveCost(location, pos):
        if location.x != pos[0] and location.y != pos[1]:
            return 1.4
        else:
            return 1

    def getCorner(next_pos,location):
        corner_cost=10
        lc_corner_cost=1000
        if(location.pre_entry==None):
            return 0
        else:
            if(location.z!=location.pre_entry.z):
                if(location.pre_entry.pre_entry==None):
                    return corner_cost
                else:
                    if(location.pre_entry.pre_entry.z != location.pre_entry.z):
                        delta_x=abs(location.pre_entry.x-next_pos[0])
                        delta_y=abs(location.pre_entry.y-next_pos[1])
                        if(delta_x==1 and delta_y==1):
                            return lc_corner_cost
                        else:
                            return 0  
                    else:
                        delta_x=abs(location.pre_entry.pre_entry.x-next_pos[0])
                        delta_y=abs(location.pre_entry.pre_entry.y-next_pos[1])
                        if(delta_x==1 and delta_y==1):
                            return lc_corner_cost
                        else:
                            return 0
            else:
                delta_x=abs(location.pre_entry.x-next_pos[0])
                delta_y=abs(location.pre_entry.y-next_pos[1])
                if(delta_x==1 and delta_y==1):
                    return corner_cost
                else:
                    return 0

                
    def getLayer(location,pos):
        layer=location.z
        change_cost=0
        layer_cost=0
        corner_or_layerc = 1
        if(location.pre_entry != None and location.pre_entry.z != location.z):
            if(location.z!=pos[2]):
                change_cost=1000
                layer_cost=1000
        elif(location.z!=pos[2]):
            change_cost=0
            layer_cost=layer_cost_func(layer)
        else:
            change_cost=0
            layer_cost=0
        return layer_cost+change_cost
    def isInList(list, pos):
        if pos in list:
            return list[pos]
        return None
    
    # add available adjacent positions
    def addAdjacentPositions(map, location, dest, openlist, closedlist):
        poslist = getPositions(map, location)
        for pos in poslist:
            # if position is already in closedlist, do nothing
            if isInList(closedlist, pos) is None:
                findEntry = isInList(openlist, pos)
                h_cost = calHeuristic(pos, dest)
                l_cost= getLayer(location,pos)
                c_cost = getCorner(pos,location)
                g_cost = location.g_cost + getMoveCost(location, pos)+c_cost+l_cost
                if findEntry is None :
                    # if position is not in openlist, add it to openlist
                    openlist[pos] = SearchEntry_3d(pos[0], pos[1],pos[2], g_cost, c_cost,g_cost+h_cost, location)
                elif findEntry.g_cost > g_cost:
                    # if position is in openlist and cost is larger than current one,
                    # then update cost and previous position
                    findEntry.g_cost = g_cost
                    findEntry.c_cost = c_cost
                    findEntry.f_cost = g_cost + h_cost
                    findEntry.pre_entry = location

    # find a least cost position in openlist, return None if openlist is empty
    def getFastPosition(openlist):
        fast = None
        for entry in openlist.values():
            #print(fast,entry)
            if fast is None:
                fast = entry
            #else:
            #    print(fast.f_cost)
            elif fast.f_cost > entry.f_cost:
                fast = entry
        return fast

    openlist = {}
    closedlist = {}
    location = SearchEntry_3d(source[0], source[1],source[2], 0.0,0.0,0.0)
    dest = SearchEntry_3d(dest[0], dest[1],dest[2], 0.0, 0.0, 0.0)
    openlist[source] = location

    while True:
        location = getFastPosition(openlist)
        #print(location)
        if location is None:
            # not found valid path
            print("can't find valid path")
            break;
        #else:
        #    print(location.x,location.y,location.z)
        if location.x == dest.x and location.y == dest.y and location.z == dest.z:
            break

        closedlist[location.getPos()] = location
        openlist.pop(location.getPos())
        addAdjacentPositions(map, location, dest, openlist, closedlist)

    #mark the found path at the map
    path_temp=[]
    while location is not None:
        #print(location.x,location.y)
        path_temp.append([location.x,location.y,location.z])
        location = location.pre_entry
    path=path_temp[::-1]
    return path



In [202]:
#测试模块
import time
map_width=30
map_height=30
map_depth=4
map2=Map_3d(map_width,map_height,map_depth)
map3=Map_3d(map_width,map_height,1)

#for i in range(map_depth):
#    map2.createBlock(200,i)
map2.createBlock(100,0)
map2.createBlock(40,1)
map2.createBlock(100,2)
#map2.createBlock(10,3)
#map2.createBlock(200,4)

#map2.createBlock(70,0)
#map2.createBlock(70,1)


#map2.setBlock([[0,1],[0,3],[1,1],[1,3],[2,1],[2,3]])
#map2.setBlock([[37,38],[36,38],[35,38],[38,38],[39,38],[33,39],[34,37],[33,37]])
#map2.setBlock([[37,1],[36,1],[35,1],[38,1],[39,1],[39,3],[37,3],[38,3],[36,3],[35,3]])

source1=(0,2,1)
dest1=(27,29,1)
#dest1=(7,9,1)
map2.clrBlock([source1,dest1])
#dest1=(37,2)

source2=(4,25,1)
dest2=(29,4,1)
map2.clrBlock([source2,dest2])
#for i in range(map_depth):
#    map2.showMap(i)
port_1=[source1[0],source1[1]]
port_2=[dest1[0],dest1[1]]

regulate_point1=regulate_route_new(port_1,3,5)#regulate端口处，source #用于mp1层 #why m-1?
regulate_point2=regulate_route_new(port_2,2,5)#regulate端口处,dest  #用于mp1层
regulate_point_3d1=regulate_route_3d(port_1,3,2)#regulate端口处，source #用于mp2 mn0层    
regulate_point_3d2=regulate_route_3d(port_2,2,2)#regulate端口处,dest  #用于mp2 mn0层   

map2.setBlock_layout(regulate_point1,1)
map2.setBlock_layout(regulate_point2,1)
map2.setBlock_layout(regulate_point_3d1,0)
map2.setBlock_layout(regulate_point_3d2,0)
map2.setBlock_layout(regulate_point_3d1,2)
map2.setBlock_layout(regulate_point_3d2,2)
#map2.showMap()
start=time.time()
path1=AStarSearch_3d(map2,source1,dest1)

for i in range(map_depth):
    map2.showMap(i)
    
map2.setBlock(path1)

#path2=AStarSearch_3d(map2,source2,dest2)
end=time.time()
print(end-start)
#print(path1)
#map2.setBlock(path2)

#print(path1)
#map2.setBlock_to2(path1)
#for i in range(map_depth):
#    map2.showMap(i)

map3.setBlock_test(path1,0)
map3.showMap(0)

#map3.setBlock_test(path2,0)
#map3.showMap(0)

#map2.showMap(0)
#map2.showMap(1)
#map2.showMap(2)


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
+          N                          N                                   0  0  0  0  0  0 +
+             N        N        N                    N                    0  0  0  0  0  0 +
+                                                                                          +
+                            N        N                       N        N                   +
+          N                                               N     N     N                   +
+                                                                N                         +
+                         N     N                                            N             +
+                N     N                       N                                   N       +
+                                                                                          +
+             N        N                       N  N                   

In [120]:
def get_multi_layer_path_type(path):
    type_list=[]
    remove_list=[] #把上下换层的重合点去掉前一个
    for i in range(len(path)-1,0,-1):
        if(path[i][0]==path[i-1][0] and path[i][1]==path[i-1][1]):
            remove_list.append(i-1)
        else:
            type_list.append(path[i][2])
    type_list=type_list[::-1]
    type_list.append(path[-1][2])
    new_path=[]
    for i in range(len(path)):
        if(i in remove_list):
            pass
        else:
            new_path.append(path[i])
        
        
    return type_list,new_path
#print(get_multi_layer_path_type(path1))
type_list,new_path=get_multi_layer_path_type(path1)

def get_multi_layer_index_list(path,first_index,last_index): #注意 这里的path是要处理掉换层重复点后的
    #根据path的坐标来获取布线path的输入输出位面，同上按十字逆时针顺序左1下2右3上4
    len_path=len(path)
    index_list=[first_index]
    for i in range(0,len_path-1):
        x_delta=path[i+1][0]-path[i][0]#获取横坐标差来判断前后两个单元是什么方向的
        y_delta=path[i+1][1]-path[i][1]#获取纵坐标差来判断前后两个单元是什么方向的
        if(x_delta==1 and y_delta==0):
            fir_to_sec=3
        elif(x_delta==-1 and y_delta==0):
            fir_to_sec=1
        elif(x_delta==0 and y_delta==1):
            fir_to_sec=4
        elif(x_delta==0 and y_delta==-1):
            fir_to_sec=2
        else:
            raise Exception("Wrong routing strategy: undefined direction -1") #理论上只有1234四个方向，如果多出来说明程序有bug
        index_list.append(fir_to_sec)
    index_list.append(last_index)
    return index_list
index_list=get_multi_layer_index_list(new_path,4,4)
print(index_list)
#print(len(new_path),len(index_list))
    
    
    
def get_multi_layer_route_type(index_seq):
    len_index=len(index_seq)
    rtype=[]
    for i in range(0,len_index-1):
        if(not index_seq[i]== index_seq[i+1]):
            rtype.append("c")#corner
            #print(i)
            #print(new_path[i-1],new_path[i],new_path[i+1])
        else:
            rtype.append("p")#path
    return rtype
rtype=get_multi_layer_route_type(index_list)
print(rtype)

IndexError: list index out of range

In [8]:
def analyze_multi_layer_path(path_coord,path_type,index_seq,rtype_seq):
    info=[]
    #drv_itface
    group_A=[1,2]
    group_B=[3,4]
    info.append([path_coord[0],index_seq[0],"mp1"])    
    #path corner cross
    len_path=len(path_coord)
    k=1
    rtype_seq_temp=[]
    for r in rtype_seq:
        rtype_seq_temp.append(r)
    #print(rtype_seq)
    for i in range(1,len_path-1):
        if(i!=len_path-2 and i!=2):
            double_corner=rtype_seq_temp[i]=="p" and rtype_seq_temp[i+1]=="c" and rtype_seq_temp[i+2]=="c" and rtype_seq_temp[i-1]=="c" and rtype_seq_temp[i-2]=="c"

        if(double_corner):
            rtype_seq[i]="c"
            #print(i)        
        elif(rtype_seq_temp[i]=="c" and rtype_seq_temp[i-1]=="p" and rtype_seq_temp[i+1]=="p"):
            #print(i)
            rtype_seq[i-1]="c"
            rtype_seq[i+1]="c"
    #print(rtype_seq)
    for i in range(1,len_path-1):    
        if(i!=len_path-2 and i!=2):
            if(rtype_seq[i-1]=="p" and rtype_seq[i]=="c" and rtype_seq[i+1]=="c" and rtype_seq[i+2]=="p" and rtype_seq[i-2]=="p"):
                rtype_seq[i-1]="c"
            elif(rtype_seq[i-1]=="p" and rtype_seq[i]=="c" and rtype_seq[i+1]=="c" and rtype_seq[i+2]=="p" and rtype_seq[i-2]=="c"):
                rtype_seq[i+2]="c"
 
    #print(rtype_seq)
    #print(path_type)
    for i in range(1,len_path-1):
        curr_path_type=path_type[k]
        layer_unchange = (path_type[i+1] == path_type[i])
        rtype_unchange = rtype_seq[i]==rtype_seq[i+1]
        info_temp=[curr_path_type,rtype_seq[k],k]
        if(i==len_path-2):#最后一个单元
            info_temp=[curr_path_type,rtype_seq[i],k]
            #print(info_temp)
            k=i 
            info_temp.append(k)
            info.append(info_temp)
            k=k+1
        elif(layer_unchange):
            if(rtype_unchange):
                pass
            else:
                #print("rtype_change at {}".format(i))
                k=i+1
                info_temp.append(k)
                info.append(info_temp)
        else:
            #print("layer_change at {}".format(i))
            k=i+1
            info_temp.append(k)
            info.append(info_temp)            
            
    #rec_itface
    info.append([path_coord[-1],index_seq[-1],"mp1"])
    
    for i in range(2,len_info-2):
        group_C=[1,3]
        group_D=[2,4]
        if(info[i][1]=="c"):
            #print(info[i])
            start=info[i][2]
            end=info[i][3]-1
            start_index=index_seq[start]
            end_index=index_seq[end+1]
            start_std_coord=std_path_coord[start]
            end_std_coord=std_path_coord[end]
            x_delta=abs(start_std_coord[0]-end_std_coord[0])
            y_delta=abs(start_std_coord[1]-end_std_coord[1])
            prev_path_length=abs(info[i-1][2]-info[i-1][3])
            next_path_length=abs(info[i+1][2]-info[i+1][3])
            if(x_delta>y_delta):     
                if(start_index in group_D and end_index in group_C):
                    info[i][3]=info[i][3]-1
                    info[i+1][2]=info[i+1][2]-1
                elif(start_index in group_C and end_index in group_D):
                    info[i][2]=info[i][2]+1
                    info[i-1][3]=info[i-1][3]+1
                elif(start_index in group_C and end_index in group_C):
                    if(info[i-1][0] in group_B):
                        info[i][3]=info[i][3]-1
                        info[i+1][2]=info[i+1][2]-1
                    elif(info[i+1][0] in group_B):
                        info[i][2]=info[i][2]+1
                        info[i-1][3]=info[i-1][3]+1
                    elif(prev_path_length>next_path_length):
                        info[i][2]=info[i][2]+1
                        info[i-1][3]=info[i-1][3]+1
                    else:
                        info[i][3]=info[i][3]-1
                        info[i+1][2]=info[i+1][2]-1
                elif(start_index in group_D and end_index in group_D):
                    if(info[i-1][0] in group_B):
                        info[i][3]=info[i][3]+1
                        info[i+1][2]=info[i+1][2]+1
                    elif(info[i+1][0] in group_B):
                        info[i][2]=info[i][2]-1
                        info[i-1][3]=info[i-1][3]-1
                    elif(prev_path_length>next_path_length):
                        info[i][2]=info[i][2]-1
                        info[i-1][3]=info[i-1][3]-1
                    else:
                        info[i][3]=info[i][3]+1
                        info[i+1][2]=info[i+1][2]+1
                
            elif(x_delta<y_delta):
                if(start_index in group_C and end_index in group_D):
                    info[i][3]=info[i][3]-1
                    info[i+1][2]=info[i+1][2]-1
                elif(start_index in group_D and end_index in group_C):
                    info[i][2]=info[i][2]+1
                    info[i-1][3]=info[i-1][3]+1
                elif(start_index in group_C and end_index in group_C):
                    if(info[i-1][0] in group_B):
                        info[i][3]=info[i][3]+1
                        info[i+1][2]=info[i+1][2]+1
                    elif(info[i+1][0] in group_B):
                        info[i][2]=info[i][2]-1
                        info[i-1][3]=info[i-1][3]-1
                    elif(prev_path_length>next_path_length):
                        info[i][2]=info[i][2]-1
                        info[i-1][3]=info[i-1][3]-1
                    else:
                        info[i][3]=info[i][3]+1
                        info[i+1][2]=info[i+1][2]+1
                elif(start_index in group_D and end_index in group_D):
                    if(info[i-1][0] in group_B):
                        info[i][3]=info[i][3]-1
                        info[i+1][2]=info[i+1][2]-1
                    elif(info[i+1][0] in group_B):
                        info[i][2]=info[i][2]+1
                        info[i-1][3]=info[i-1][3]+1
                    elif(prev_path_length>next_path_length):
                        info[i][2]=info[i][2]+1
                        info[i-1][3]=info[i-1][3]+1
                    else:
                        info[i][3]=info[i][3]-1
                        info[i+1][2]=info[i+1][2]-1


    return info

def get_multi_layer_path_info(path):
    path_2d=[]
    path_type=[]
    for p in path:
        path_2d.append([p[0],p[1]])
        path_type.append(p[2])
    return path_2d,path_type
path_2d,path_type=get_multi_layer_path_info(new_path)
info=analyze_multi_layer_path(path_2d,path_type,index_list,rtype)
print(info,end="\n\n")
print(path_2d,end="\n\n")
print(path_type,end="\n\n")

[[[0, 2], 4, 'mp1'], [2, 'p', 1, 3], [3, 'p', 3, 5], [4, 'p', 5, 8], [3, 'p', 8, 10], [2, 'p', 10, 12], [3, 'p', 12, 21], [3, 'c', 21, 24], [3, 'p', 24, 26], [2, 'p', 26, 29], [3, 'p', 29, 34], [2, 'p', 34, 45], [1, 'p', 45, 48], [1, 'c', 48, 51], [1, 'p', 51, 53], [[27, 29], 4, 'mp1']]

[[0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [0, 14], [0, 15], [0, 16], [0, 17], [0, 18], [0, 19], [0, 20], [0, 21], [0, 22], [0, 23], [0, 24], [1, 24], [2, 24], [3, 24], [4, 24], [5, 24], [6, 24], [7, 24], [8, 24], [9, 24], [10, 24], [11, 24], [12, 24], [13, 24], [14, 24], [15, 24], [16, 24], [17, 24], [18, 24], [19, 24], [20, 24], [21, 24], [22, 24], [23, 24], [24, 24], [25, 24], [26, 24], [27, 24], [27, 25], [27, 26], [27, 27], [27, 28], [27, 29]]

[1, 2, 2, 3, 3, 4, 4, 4, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



In [16]:
def last_check_index(index):#替换掉目标port的位置信息
    if(index==1):#这个函数是为了弥补之前程序设计不合理，把终点端口的位面换成和排线方向一致的值
        last_index=3 #（相对于上一个单元来说，如果终点端口位于1上一个line单元必须是3方向输出）
    elif(index==2):
        last_index=4;
    elif(index==3):
        last_index=1;
    elif(index==4):
        last_index=2;
    return last_index

def get_descend(index,source,dest):#用path的输入输出坐标来判断布线的走向是上浮还是下沉
    if(index==1 or index==3):
        if(source[1]-dest[1]>0):
            descend=1
        else:
            descend=0
    elif(index==2 or index==4):
        if(source[0]-dest[0]>0):
            descend=1
        else:
            descend=0
    return descend

#------------添加到param_lib中-----------------
def index_to_layer_multi_layer(index):
    layer=["mn0","mp1","mp2","rn0","cn0","rn0","cn0"]
    return layer[index]
#-------------------------------------------------------
layout_unit_len=30
drv_out=4
wire_width=14
rec_out=8
enlarge_coef=2
def multi_layer_path_to_pcell(info,path_coord,index_seq):
    #获取布线信息长度
    len_info=len(info)
    #print(len_info)
    #开始添加SKILL脚本
    script=[]
    first_itface=interface(drv_out,wire_width,path_coord[0],info[0][1],info[0][2])
    script.append(first_itface)
      
    
    for i in range(1,len_info-1):
        if(info[i][1]=="p"):              
            if(i==1):#处理第一个path
                if(info[i][0]!=1):#如果第一个path不是mp1层的  #注 这里cross的上下层关系需要加判断
                    cross_i=cross_multi_layer(wire_width,path_coord[info[i][2]],index_seq[info[i][2]],last_check_index(index_seq[info[i][2]]),1,info[i][0])
                    path_i=path(path_coord[info[i][2]+1],path_coord[info[i][3]],wire_width,index_to_layer_multi_layer(info[i][0]))
                    script.append(cross_i)
                else:#如果是 不添加cros
                    path_i=path(path_coord[info[i][2]],path_coord[info[i][3]],wire_width,index_to_layer_multi_layer(info[i][0]))
            else:#处理第二个及以后的path
                if(info[i][0]!=info[i-1][0]):#如果和上一个path不是同一层
                    cross_i=cross_multi_layer(wire_width,path_coord[info[i][2]],index_seq[info[i][2]],last_check_index(index_seq[info[i][2]]),info[i-1][0],info[i][0])
                    script.append(cross_i)
                    path_i=path(path_coord[info[i][2]+1],path_coord[info[i][3]],wire_width,index_to_layer_multi_layer(info[i][0]))
                else:#如果是同一层
                    path_i=path(path_coord[info[i][2]],path_coord[info[i][3]],wire_width,index_to_layer_multi_layer(info[i][0]))
            script.append(path_i)
        elif(info[i][1]=="c"):
            group_C=[1,3]
            group_D=[2,4]
            same_index= (index_seq[info[i][3]] in group_C) == (index_seq[info[i][2]] in group_C)

            if(i==len_info-2):
                index_temp=info[i][3]
            else:
                index_temp=info[i][3]-1
            same_index= (index_seq[index_temp] in group_C) == (index_seq[info[i][2]] in group_C)

            if(index_seq[info[i][2]]==1):
                if(same_index):
                    corner_width=(abs(path_coord[index_temp][0]-path_coord[info[i][2]][0])+layout_unit_len)/layout_unit_len
                else:
                    corner_width=(abs(path_coord[index_temp][0]-path_coord[info[i][2]][0])+layout_unit_len/2)/layout_unit_len

            elif(index_seq[info[i][2]]==2):
                if(same_index):
                    corner_width=(abs(path_coord[index_temp][0]-path_coord[info[i][2]][0])+layout_unit_len)/layout_unit_len
                else:
                    corner_width=(abs(path_coord[index_temp][0]-path_coord[info[i][2]][0])+3*layout_unit_len/2)/layout_unit_len
            elif(index_seq[info[i][2]]==3):
                if(same_index):
                    corner_width=(abs(path_coord[index_temp][0]-path_coord[info[i][2]][0])+layout_unit_len)/layout_unit_len
                else:
                    corner_width=(abs(path_coord[index_temp][0]-path_coord[info[i][2]][0])+layout_unit_len/2)/layout_unit_len
            elif(index_seq[info[i][2]]==4):
                if(same_index):
                    corner_width=(abs(path_coord[index_temp][0]-path_coord[info[i][2]][0])+layout_unit_len)/layout_unit_len
                else:
                    corner_width=(abs(path_coord[index_temp][0]-path_coord[info[i][2]][0])+3*layout_unit_len/2)/layout_unit_len
            else:
                return ["print('failed')"]
            #print(corner_width)
            wire_type=index_to_layer_multi_layer(info[i][0])
            origin=path_coord[info[i][2]]
            in_index=index_seq[info[i][2]]
            out_index=index_seq[info[i][3]+1]
            descend=get_descend(in_index,path_coord[info[i][2]],path_coord[info[i][3]])
            corner_i=corner(wire_width,corner_width,wire_type,origin,in_index,out_index,descend)
            script.append(corner_i)
        else:
            return ["print('failed')"]
    last_index=last_check_index(info[-1][1])
    if(last_index==1): #末尾的rec interface坐标调整
        coord=[path_coord[-1][0]-layout_unit_len,path_coord[-1][1]]
    elif(last_index==2):
        coord=[path_coord[-1][0],path_coord[-1][1]-layout_unit_len]
    elif(last_index==3):
        coord=[path_coord[-1][0]+layout_unit_len,path_coord[-1][1]]
    elif(last_index==4):
        coord=[path_coord[-1][0],path_coord[-1][1]+layout_unit_len]   
    last_itface=interface(rec_out,wire_width,coord,last_index,info[-1][2])
    script.append(last_itface)
    return script
def pcell_coord(coord,in_index):#把path的原点（默认在左下角）转到输入位面的中点
    if(in_index==1):
        origin=[coord[0]+layout_unit_len,coord[1]+layout_unit_len/2]
    elif(in_index==2):
        origin=[coord[0]+layout_unit_len/2,coord[1]+layout_unit_len]
    elif(in_index==3):
        origin=[coord[0],coord[1]+layout_unit_len/2]
    elif(in_index==4):
        origin=[coord[0]+layout_unit_len/2,coord[1]]    
    return origin
        
type_list,new_path=get_multi_layer_path_type(path1)
    
std_path_coord=[]
len_path=len(new_path)
for p in range(0,len_path-1):#转为pcell坐标
    new_path[p]=[new_path[p][0]*layout_unit_len,new_path[p][1]*layout_unit_len]
    std_path_coord.append(new_path[p])
    new_path[p]=pcell_coord(new_path[p],index_list[p])
#设置最后一个点的pcell坐标
new_path[-1]=[new_path[-1][0]*layout_unit_len,new_path[-1][1]*layout_unit_len]
std_path_coord.append(new_path[-1])
new_path[-1]=pcell_coord(new_path[-1],last_check_index(index_list[-1]))
script=multi_layer_path_to_pcell(info,new_path,index_list)
for s in script:
    print(s)


ref=rodCreatePolygon(?cvId cellID ?layer "mn0" ?pts list(17.0:60 21.5:64.5 21.5:66.5 8.5:66.5 8.5:64.5 13.0:60))
rodCreatePolygon(?cvId cellID ?layer "in0" ?fromObj ref ?size -0.5)
rodCreatePolygon(?cvId cellID ?layer "mp1" ?fromObj ref ?size 0.5)
rodCreatePath(?layer "mp1" ?pts list(15.0:67.0 15.0:90) ?width 14 ?justification "center" ?cvId  cellID)

ref=rodCreatePolygon(?cvId cellID ?layer "mp1" ?pts list(8.0:98.0 8.0:112.0 22.0:112.0 22.0:98.0))
rodCreatePolygon(?cvId cellID ?layer "in0" ?fromObj ref ?size -1)
rodCreatePolygon(?cvId cellID ?layer "mp2" ?fromObj ref ?size 1)
rodCreatePath(?layer "mp1" ?pts list(15.0:98.0 15.0:90.0) ?width 14 ?justification "center" ?cvId  cellID)
rodCreatePath(?layer "mp2" ?pts list(15.0:120.0 15.0:113.0) ?width 14 ?justification "center" ?cvId  cellID)

rodCreatePath(?layer "mp2" ?pts list(15.0:120 15.0:150) ?width 14 ?justification "center" ?cvId  cellID)

ref=rodCreatePolygon(?cvId cellID ?layer "mp2" ?pts list(8.0:158.0 8.0:172.0 22.0:172.0 22.0

In [10]:
def interface(input_width,output_width,origin,rotate,layer):#interface的python模型 #origin为30*30方格的0,15处
    path_width=output_width
    if(layer=="mp1"):
        output_width=output_width-1
        itface_width=(output_width-input_width)/2+enlarge_coef+0.5
    else:
        itface_width=(output_width-input_width)/2+enlarge_coef
    if(rotate==1):
        p1=[origin[0],origin[1]+input_width/2]
        p2=[origin[0]-(output_width-input_width)/2,origin[1]+output_width/2]
        p3=[p2[0]-enlarge_coef,p2[1]]
        p4=[p3[0],origin[1]-output_width/2]
        p5=[origin[0]-(output_width-input_width)/2,origin[1]-output_width/2]
        p6=[origin[0],origin[1]-input_width/2]
        path_1=[origin[0]-itface_width,origin[1]]
        path_2=[origin[0]-layout_unit_len,origin[1]]
    elif(rotate==2):
        p1=[origin[0]+input_width/2,origin[1]]
        p2=[origin[0]+output_width/2,origin[1]-(output_width-input_width)/2]
        p3=[p2[0],p2[1]-enlarge_coef]
        p4=[origin[0]-output_width/2,p3[1]]
        p5=[origin[0]-output_width/2,origin[1]-(output_width-input_width)/2]
        p6=[origin[0]-input_width/2,origin[1]]
        path_1=[origin[0],origin[1]-itface_width]
        path_2=[origin[0],origin[1]-layout_unit_len]
    elif(rotate==3):
        p1=[origin[0],origin[1]+input_width/2]
        p2=[origin[0]+(output_width-input_width)/2,origin[1]+output_width/2]
        p3=[p2[0]+enlarge_coef,p2[1]]
        p4=[p3[0],origin[1]-output_width/2]
        p5=[origin[0]+(output_width-input_width)/2,origin[1]-output_width/2]
        p6=[origin[0],origin[1]-input_width/2]
        path_1=[origin[0]+itface_width,origin[1]]
        path_2=[origin[0]+layout_unit_len,origin[1]]
    elif(rotate==4):
        p1=[origin[0]+input_width/2,origin[1]]
        p2=[origin[0]+output_width/2,origin[1]+(output_width-input_width)/2]
        p3=[p2[0],p2[1]+enlarge_coef]
        p4=[origin[0]-output_width/2,p3[1]]
        p5=[origin[0]-output_width/2,origin[1]+(output_width-input_width)/2]
        p6=[origin[0]-input_width/2,origin[1]]
        path_1=[origin[0],origin[1]+itface_width]
        path_2=[origin[0],origin[1]+layout_unit_len]
    if(layer=="mp1"):
        script='''
ref=rodCreatePolygon(?cvId cellID ?layer "mn0" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11}))
rodCreatePolygon(?cvId cellID ?layer "in0" ?fromObj ref ?size -0.5)
rodCreatePolygon(?cvId cellID ?layer "mp1" ?fromObj ref ?size 0.5)
rodCreatePath(?layer "mp1" ?pts list({12}:{13} {14}:{15}) ?width {16} ?justification "center" ?cvId  cellID)'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],path_1[0],path_1[1],path_2[0],path_2[1],path_width)
    elif(layer=="mn0"):
        script='''
ref=rodCreatePolygon(?cvId cellID ?layer "mn0" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11}))
rodCreatePath(?layer "mn0" ?pts list({12}:{13} {14}:{15}) ?width {16} ?justification "center" ?cvId  cellID)'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],path_1[0],path_1[1],path_2[0],path_2[1],output_width)
    else:
        script='''
ref=rodCreatePolygon(?cvId cellID ?layer "{17}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11}))
rodCreatePath(?layer "{18}" ?pts list({12}:{13} {14}:{15}) ?width {16} ?justification "center" ?cvId  cellID)'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],path_1[0],path_1[1],path_2[0],path_2[1],output_width,layer,layer)
    return script

In [11]:
def corner(wire_width,corner_width,wire_type,origin,in_index,out_index,descend):#corner的python模型，descend为 13输入则向下，24输入则向左 #origin为30*30方格的0,15处
    tan675=2.414213562
    x=layout_unit_len/2-wire_width/(2*tan675)
    if(in_index==1):
        if(out_index==2):
            p1=[origin[0],origin[1]+wire_width/2]
            p2=[origin[0]-wire_width/tan675,origin[1]+wire_width/2]
            p3=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/2,origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/tan675]
            p4=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/2,origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)]
            p5=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/2,origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)]
            p6=[origin[0],origin[1]-wire_width/2]
            script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type)
        elif(out_index==1):
            if(descend==1):
                p1=[origin[0],origin[1]+wire_width/2]
                p2=[origin[0]-wire_width/tan675-x,origin[1]+wire_width/2]
                p3=[origin[0]-corner_width*layout_unit_len+x,origin[1]-(corner_width-1)*layout_unit_len+wire_width/2]
                p4=[origin[0]-corner_width*layout_unit_len,origin[1]-(corner_width-1)*layout_unit_len+wire_width/2]                
                p5=[origin[0]-corner_width*layout_unit_len,origin[1]-(corner_width-1)*layout_unit_len-wire_width/2]
                p6=[origin[0]-corner_width*layout_unit_len+x+wire_width/tan675,origin[1]-(corner_width-1)*layout_unit_len-wire_width/2]
                p7=[origin[0]-x,origin[1]-wire_width/2]
                p8=[origin[0],origin[1]-wire_width/2]
                script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11} {13}:{14} {15}:{16}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type,p7[0],p7[1],p8[0],p8[1])
            elif(descend==0):
                p1=[origin[0],origin[1]-wire_width/2]
                p2=[origin[0]-wire_width/tan675-x,origin[1]-wire_width/2]
                p3=[origin[0]-corner_width*layout_unit_len+x,origin[1]+(corner_width-1)*layout_unit_len-wire_width/2]
                p4=[origin[0]-corner_width*layout_unit_len,origin[1]+(corner_width-1)*layout_unit_len-wire_width/2]                
                p5=[origin[0]-corner_width*layout_unit_len,origin[1]+(corner_width-1)*layout_unit_len+wire_width/2]
                p6=[origin[0]-corner_width*layout_unit_len+x+wire_width/tan675,origin[1]+(corner_width-1)*layout_unit_len+wire_width/2]
                p7=[origin[0]-x,origin[1]+wire_width/2]
                p8=[origin[0],origin[1]+wire_width/2]
                script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11} {13}:{14} {15}:{16}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type,p7[0],p7[1],p8[0],p8[1])
        elif(out_index==4):
            p1=[origin[0],origin[1]-wire_width/2]
            p2=[origin[0]-wire_width/tan675,origin[1]-wire_width/2]
            p3=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/2,origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/tan675]
            p4=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/2,origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)]
            p5=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/2,origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)]
            p6=[origin[0],origin[1]+wire_width/2]
            script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type)
            
    elif(in_index==2):
        if(out_index==1):
            p1=[origin[0]+wire_width/2,origin[1]]
            p2=[origin[0]+wire_width/2,origin[1]-wire_width/tan675]
            p3=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/tan675,origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/2]
            p4=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2),origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/2]
            p5=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2),origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/2]
            p6=[origin[0]-wire_width/2,origin[1]]
            script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type)
        elif(out_index==2):
            if(descend==1):
                p1=[origin[0]+wire_width/2,origin[1]]
                p2=[origin[0]+wire_width/2,origin[1]-wire_width/tan675-x]
                p3=[origin[0]-(corner_width-1)*layout_unit_len+wire_width/2,origin[1]-corner_width*layout_unit_len+x]
                p4=[origin[0]-(corner_width-1)*layout_unit_len+wire_width/2,origin[1]-corner_width*layout_unit_len]                
                p5=[origin[0]-(corner_width-1)*layout_unit_len-wire_width/2,origin[1]-corner_width*layout_unit_len]
                p6=[origin[0]-(corner_width-1)*layout_unit_len-wire_width/2,origin[1]-corner_width*layout_unit_len+x+wire_width/tan675]
                p7=[origin[0]-wire_width/2,origin[1]-x]
                p8=[origin[0]-wire_width/2,origin[1]]
                script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11} {13}:{14} {15}:{16}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type,p7[0],p7[1],p8[0],p8[1])
            elif(descend==0):
                p1=[origin[0]-wire_width/2,origin[1]]
                p2=[origin[0]-wire_width/2,origin[1]-wire_width/tan675-x]
                p3=[origin[0]+(corner_width-1)*layout_unit_len-wire_width/2,origin[1]-corner_width*layout_unit_len+x]
                p4=[origin[0]+(corner_width-1)*layout_unit_len-wire_width/2,origin[1]-corner_width*layout_unit_len]                
                p5=[origin[0]+(corner_width-1)*layout_unit_len+wire_width/2,origin[1]-corner_width*layout_unit_len]
                p6=[origin[0]+(corner_width-1)*layout_unit_len+wire_width/2,origin[1]-corner_width*layout_unit_len+x+wire_width/tan675]
                p7=[origin[0]+wire_width/2,origin[1]-x]
                p8=[origin[0]+wire_width/2,origin[1]]
                script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11} {13}:{14} {15}:{16}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type,p7[0],p7[1],p8[0],p8[1])
        elif(out_index==3):
            p1=[origin[0]-wire_width/2,origin[1]]
            p2=[origin[0]-wire_width/2,origin[1]-wire_width/tan675]
            p3=[origin[0]+((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/tan675,origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/2]
            p4=[origin[0]+((corner_width-1)*layout_unit_len+layout_unit_len/2),origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/2]
            p5=[origin[0]+((corner_width-1)*layout_unit_len+layout_unit_len/2),origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/2]
            p6=[origin[0]+wire_width/2,origin[1]]
            script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type)
            
    elif(in_index==3):
        if(out_index==2):
            p1=[origin[0],origin[1]+wire_width/2]
            p2=[origin[0]+wire_width/tan675,origin[1]+wire_width/2]
            p3=[origin[0]+(corner_width-1)*layout_unit_len+layout_unit_len/2+wire_width/2,origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/tan675]
            p4=[origin[0]+(corner_width-1)*layout_unit_len+layout_unit_len/2+wire_width/2,origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)]
            p5=[origin[0]+(corner_width-1)*layout_unit_len+layout_unit_len/2-wire_width/2,origin[1]-((corner_width-1)*layout_unit_len+layout_unit_len/2)]
            p6=[origin[0],origin[1]-wire_width/2]
            script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type)
        elif(out_index==3):
            if(descend==1):
                p1=[origin[0],origin[1]+wire_width/2]
                p2=[origin[0]+wire_width/tan675+x,origin[1]+wire_width/2]
                p3=[origin[0]+corner_width*layout_unit_len-x,origin[1]-(corner_width-1)*layout_unit_len+wire_width/2]
                p4=[origin[0]+corner_width*layout_unit_len,origin[1]-(corner_width-1)*layout_unit_len+wire_width/2]                
                p5=[origin[0]+corner_width*layout_unit_len,origin[1]-(corner_width-1)*layout_unit_len-wire_width/2]
                p6=[origin[0]+corner_width*layout_unit_len-x-wire_width/tan675,origin[1]-(corner_width-1)*layout_unit_len-wire_width/2]
                p7=[origin[0]+x,origin[1]-wire_width/2]
                p8=[origin[0],origin[1]-wire_width/2]
                script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11} {13}:{14} {15}:{16}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type,p7[0],p7[1],p8[0],p8[1])
            elif(descend==0):
                p1=[origin[0],origin[1]-wire_width/2]
                p2=[origin[0]+wire_width/tan675+x,origin[1]-wire_width/2]
                p3=[origin[0]+corner_width*layout_unit_len-x,origin[1]+(corner_width-1)*layout_unit_len-wire_width/2]
                p4=[origin[0]+corner_width*layout_unit_len,origin[1]+(corner_width-1)*layout_unit_len-wire_width/2]                
                p5=[origin[0]+corner_width*layout_unit_len,origin[1]+(corner_width-1)*layout_unit_len+wire_width/2]
                p6=[origin[0]+corner_width*layout_unit_len-x-wire_width/tan675,origin[1]+(corner_width-1)*layout_unit_len+wire_width/2]
                p7=[origin[0]+x,origin[1]+wire_width/2]
                p8=[origin[0],origin[1]+wire_width/2]
                script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11} {13}:{14} {15}:{16}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type,p7[0],p7[1],p8[0],p8[1])
        elif(out_index==4):
            p1=[origin[0],origin[1]-wire_width/2]
            p2=[origin[0]+wire_width/tan675,origin[1]-wire_width/2]
            p3=[origin[0]+((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/2,origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/tan675]
            p4=[origin[0]+((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/2,origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)]
            p5=[origin[0]+((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/2,origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)]
            p6=[origin[0],origin[1]+wire_width/2]
            script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type)

    elif(in_index==4):
        if(out_index==1):
            p1=[origin[0]+wire_width/2,origin[1]]
            p2=[origin[0]+wire_width/2,origin[1]+wire_width/tan675]
            p3=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/tan675,origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/2]
            p4=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2),origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/2]
            p5=[origin[0]-((corner_width-1)*layout_unit_len+layout_unit_len/2),origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/2]
            p6=[origin[0]-wire_width/2,origin[1]]
            script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type)
        elif(out_index==4):
            if(descend==1):
                p1=[origin[0]+wire_width/2,origin[1]]
                p2=[origin[0]+wire_width/2,origin[1]+wire_width/tan675+x]
                p3=[origin[0]-(corner_width-1)*layout_unit_len+wire_width/2,origin[1]+corner_width*layout_unit_len-x]
                p4=[origin[0]-(corner_width-1)*layout_unit_len+wire_width/2,origin[1]+corner_width*layout_unit_len]                
                p5=[origin[0]-(corner_width-1)*layout_unit_len-wire_width/2,origin[1]+corner_width*layout_unit_len]
                p6=[origin[0]-(corner_width-1)*layout_unit_len-wire_width/2,origin[1]+corner_width*layout_unit_len-x-wire_width/tan675]
                p7=[origin[0]-wire_width/2,origin[1]+x]
                p8=[origin[0]-wire_width/2,origin[1]]
                script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11} {13}:{14} {15}:{16}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type,p7[0],p7[1],p8[0],p8[1])
            elif(descend==0):
                p1=[origin[0]-wire_width/2,origin[1]]
                p2=[origin[0]-wire_width/2,origin[1]+wire_width/tan675+x]
                p3=[origin[0]+(corner_width-1)*layout_unit_len-wire_width/2,origin[1]+corner_width*layout_unit_len-x]
                p4=[origin[0]+(corner_width-1)*layout_unit_len-wire_width/2,origin[1]+corner_width*layout_unit_len]                
                p5=[origin[0]+(corner_width-1)*layout_unit_len+wire_width/2,origin[1]+corner_width*layout_unit_len]
                p6=[origin[0]+(corner_width-1)*layout_unit_len+wire_width/2,origin[1]+corner_width*layout_unit_len-x-wire_width/tan675]
                p7=[origin[0]+wire_width/2,origin[1]+x]
                p8=[origin[0]+wire_width/2,origin[1]]
                script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11} {13}:{14} {15}:{16}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type,p7[0],p7[1],p8[0],p8[1])
        elif(out_index==3):
            p1=[origin[0]-wire_width/2,origin[1]]
            p2=[origin[0]-wire_width/2,origin[1]+wire_width/tan675]
            p3=[origin[0]+((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/tan675,origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/2]
            p4=[origin[0]+((corner_width-1)*layout_unit_len+layout_unit_len/2),origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)+wire_width/2]
            p5=[origin[0]+((corner_width-1)*layout_unit_len+layout_unit_len/2),origin[1]+((corner_width-1)*layout_unit_len+layout_unit_len/2)-wire_width/2]
            p6=[origin[0]+wire_width/2,origin[1]]
            script='''
rodCreatePolygon(?cvId cellID ?layer "{12}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7} {8}:{9} {10}:{11}))'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],p5[0],p5[1],p6[0],p6[1],wire_type)
            
    return script

In [12]:
def path(source,dest,width,wire_type):#普通线的模型 #origin为线的中心
    script='''
rodCreatePath(?layer "{0}" ?pts list({2}:{3} {4}:{5}) ?width {1} ?justification "center" ?cvId  cellID)'''.format(wire_type,width,source[0],source[1],dest[0],dest[1])
    return script


In [15]:
def cross_multi_layer(width,origin,in_index,ctype,l1,l2):#cross的模型 #origin为30*30方格的0,15处
    if(l1<l2):
        lh=True
    else:
        lh=False
    if(in_index==1):
        center=[origin[0]-layout_unit_len/2,origin[1]]
    elif(in_index==2):
        center=[origin[0],origin[1]-layout_unit_len/2]
    elif(in_index==3):
        center=[origin[0]+layout_unit_len/2,origin[1]]
    elif(in_index==4):
        center=[origin[0],origin[1]+layout_unit_len/2]
    p1=[center[0]-width/2,center[1]-width/2]
    p2=[center[0]-width/2,center[1]+width/2]
    p3=[center[0]+width/2,center[1]+width/2]
    p4=[center[0]+width/2,center[1]-width/2]

    if(not lh):
        layer1=index_to_layer_multi_layer(l1)
        layer2=index_to_layer_multi_layer(l2)
    else:
        layer1=index_to_layer_multi_layer(l2)
        layer2=index_to_layer_multi_layer(l1)
    script='''
ref=rodCreatePolygon(?cvId cellID ?layer "{8}" ?pts list({0}:{1} {2}:{3} {4}:{5} {6}:{7}))
rodCreatePolygon(?cvId cellID ?layer "in0" ?fromObj ref ?size -1)
rodCreatePolygon(?cvId cellID ?layer "{9}" ?fromObj ref ?size 1)'''.format(p1[0],p1[1],p2[0],p2[1],p3[0],p3[1],p4[0],p4[1],layer2,layer1)
    if(ctype==1):
        if(not lh):
            ad_path1=path([center[0]-layout_unit_len/2,center[1]],[center[0]-width/2-1,center[1]],width,layer1)
            ad_path2=path([center[0]+width/2,center[1]],[center[0]+layout_unit_len/2,center[1]],width,layer2)
        else:
            ad_path1=path([center[0]-width/2,center[1]],[center[0]-layout_unit_len/2,center[1]],width,layer2)
            ad_path2=path([center[0]+layout_unit_len/2,center[1]],[center[0]+width/2+1,center[1]],width,layer1)
    elif(ctype==2):
        if(not lh):
            ad_path1=path([center[0],center[1]-layout_unit_len/2],[center[0],center[1]-width/2-1],width,layer1)
            ad_path2=path([center[0],center[1]+width/2],[center[0],center[1]+layout_unit_len/2],width,layer2)
        else:
            ad_path1=path([center[0],center[1]-width/2],[center[0],center[1]-layout_unit_len/2],width,layer2)
            ad_path2=path([center[0],center[1]+layout_unit_len/2],[center[0],center[1]+width/2+1],width,layer1)
    elif(ctype==3):
        if(not lh):
            ad_path1=path([center[0]+width/2+1,center[1]],[center[0]+layout_unit_len/2,center[1]],width,layer1)
            ad_path2=path([center[0]-layout_unit_len/2,center[1]],[center[0]-width/2,center[1]],width,layer2)
        else:
            ad_path1=path([center[0]-width/2-1,center[1]],[center[0]-layout_unit_len/2,center[1]],width,layer1)
            ad_path2=path([center[0]+layout_unit_len/2,center[1]],[center[0]+width/2,center[1]],width,layer2)           
    elif(ctype==4):
        if(not lh):
            ad_path1=path([center[0],center[1]+width/2+1],[center[0],center[1]+layout_unit_len/2],width,layer1)
            ad_path2=path([center[0],center[1]-layout_unit_len/2],[center[0],center[1]-width/2],width,layer2)
        else:
            ad_path1=path([center[0],center[1]+layout_unit_len/2],[center[0],center[1]+width/2],width,layer2)
            ad_path2=path([center[0],center[1]-width/2-1],[center[0],center[1]-layout_unit_len/2],width,layer1)
    return script+ad_path1+ad_path2

In [ ]:
def cross_map_search_3d(info,lib,cell,display): #3d A*版本
    #布线信息列表
    width=info[0]#获取map最大宽度
    height=info[1]#获取map最大高度
    depth=len(total_layer)#从param_lib中获取层数信息
    layout_info=info[2]#获取layout版图信息
    len_layout=len(layout_info)
    map_origin=info[3]#获取layout原点
    coord_info=info[4]#获取互联线坐标
    connection_info=info[5]#获取互联线信息
    range_list=info[6]
    #info_coord=open("routing_coord_info","w")
    #print(connection_info,file=info_coord)
    #print(coord_info,file=info_coord)
    coord_len=len(coord_info)
    #map列表
    layout_map=Map_3d(width,height,depth) #总map

    layout_map_reg_mp1=Map(width,height)#记录block info
    layout_map_reg_mp2=Map(width,height)#记录block info
    layout_map_reg_mn0=Map(width,height)#记录block info
    
    
    def showMap_all(layout_map):#显示所有map函数
        if(display==1):
            for k in range(0,depth):
                layout_map.showMap(k)
            
    text=open("./createinst.il",'a+')
    cellid='''cellID=dbOpenCellViewByType("{0}" "{1}" "layout" "maskLayout")'''.format(lib,cell)#dbopen函数，指定单元
    print(cellid,file=text)
    #showMap_all(map_list)
    for i in range(0,len_layout):#把版图中的单元作为block放置到map中 #注意 ptl层不做此设置
        block_point=origin_to_blockpoint_enlarged(layout_info[i].area,layout_info[i].xy,layout_info[i].orient)
        abs_point=get_abs_block_point(map_origin,block_point)
        #print(abs_point)
        for j in range(0,depth-len(ptl_layer)):#注意 只对mn0-mp2做此设置
            layout_map.setBlock_layout(abs_point,j)


    for i in range(0,coord_len):#封锁端口，先把所有端口都封上，在布每一条线的时候再单独打开（clrBlock函数）
        port_1_temp=port_coord_to_map(coord_info[i][0][0],coord_info[i][0][1])#获取第一个端口坐标 map中的
        port_1=[port_1_temp[0]-map_origin[0],port_1_temp[1]-map_origin[1]]#获取第一个端口的绝对坐标 map中的
        port_2_temp=port_coord_to_map(coord_info[i][1][0],coord_info[i][1][1])#同上
        port_2=[port_2_temp[0]-map_origin[0],port_2_temp[1]-map_origin[1]]#同上上
        block_source=[port_2[0],port_2[1]]#获取出发点
        block_dest=[port_1[0],port_1[1]]#获取终点
        layout_map.setBlock_layout([block_source,block_dest],1)#mp1层封锁端口

    layout_map.copyMap(layout_map_reg_mn0,0)
    layout_map.copyMap(layout_map_reg_mp1,1)
    layout_map.copyMap(layout_map_reg_mp2,2)
        
    script_summary=[]#il文件暂存列表
    failed_path_summary=[]#失败的路径index信息        
    util_summary=[]#每次布线得出的最终map利用率
    log_summary=[]#输出到logger中的信息                
                
    def run_routing(start):#从start到尾的布线函数
        script_all=[]#每次routing时产生的script
        failed_path=[]#每次routing是产生的失败路径
        log_failed_coord=[]
        for i in range(start,coord_len):#注意这里的coord和layout_lib中的某些设置不一样，在后面temp中重新排布
            #print(coord_info[i])
            port_1_temp=port_coord_to_map(coord_info[i][0][0],coord_info[i][0][1])#获取第一个端口坐标 map中的
            port_1=[port_1_temp[0]-map_origin[0],port_1_temp[1]-map_origin[1]]#获取第一个端口的绝对坐标 map中的
            port_2_temp=port_coord_to_map(coord_info[i][1][0],coord_info[i][1][1])#同上
            port_2=[port_2_temp[0]-map_origin[0],port_2_temp[1]-map_origin[1]]#同上上
            source=(port_2[0],port_2[1],1)#获取出发点
            dest=(port_1[0],port_1[1],1)#获取终点
            
            layout_map.clrBlock_layout([[port_2[0],port_2[1]],[port_1[0],port_1[1]]],1)#mp1层清除端口

            #------------get regulator lenth-----------------
            max_port_reg_try=7#暂定，增加这个值会增大布线所用时间
            min_port_reg_try=2#最小必须为2，保证regulator长度为3，confirmed
            x_delta=port_2[0]-port_1[0]#以下程序段获取出发点和终点的坐标差，来确定端口处保持多长的直线路径
            y_delta=port_2[1]-port_1[1]
            horizon_index = coord_info[i][0][1] in [1,3] and coord_info[i][1][1] in [1,3]
            vertical_index = coord_info[i][0][1] in [2,4] and coord_info[i][1][1] in [2,4]
            if(horizon_index):
                max_port_reg_try_temp=abs(int(x_delta/2))-1
            elif(vertical_index):
                max_port_reg_try_temp=abs(int(y_delta/2))-1
            else:
                max_port_reg_try_temp=min([abs(int(x_delta/2))-1,abs(int(y_delta/2))-1])
            if(max_port_reg_try_temp<max_port_reg_try and max_port_reg_try_temp>=3):
                max_port_reg_try=max_port_reg_try_temp
            elif(max_port_reg_try_temp<3):
                max_port_reg_try=3
            #---------------------------------------------
            for m in range(max_port_reg_try,min_port_reg_try,-1):#iteration开始
                print("iteration {0} at path {1}".format(max_port_reg_try-m+1,i+1))
                regulate_point1=regulate_route_new(port_1,coord_info[i][0][1],m)#regulate端口处，source #用于mp1层
                regulate_point2=regulate_route_new(port_2,coord_info[i][1][1],m)#regulate端口处,dest  #用于mp1层
                regulate_point_3d1=regulate_route_3d(port_1,coord_info[i][0][1],m)#regulate端口处，source #用于mp2 mn0层    
                regulate_point_3d2=regulate_route_3d(port_2,coord_info[i][1][1],m)#regulate端口处,dest  #用于mp2 mn0层   
                #暂存map中原来的数据
                prev_mn0_source=[]
                prev_mp1_source=[]
                prev_mp2_source=[]
                prev_mn0_dest=[]
                prev_mp1_dest=[]
                prev_mp2_dest=[]

                min_path=[]
                #获取原来的数据，记录下后设置port regulator
                #------------------处理mp1------------------------------------
                prev_mp1_block_1=layout_map.get_blocked_point(regulate_point1,1) #获取mp1的source处原来被屏蔽的点
                prev_mp1_block_2=layout_map.get_blocked_point(regulate_point2,1) #获取mp1的dest处原来被屏蔽的点
                prev_mp1_source.append(prev_mp1_block_1)
                prev_mp1_dest.append(prev_mp1_block_2)
                layout_map.setBlock_layout(regulate_point1,1)
                layout_map.setBlock_layout(regulate_point2,1)
                #------------------------------处理mn0-------------
                prev_mn0_block_1=layout_map.get_blocked_point(regulate_point_3d1,0) #获取mn0的source处原来被屏蔽的点
                prev_mn0_block_2=layout_map.get_blocked_point(regulate_point_3d2,0) #获取mn0的dest处原来被屏蔽的点
                prev_mn0_source.append(prev_mn0_block_1)
                prev_mn0_dest.append(prev_mn0_block_2)
                layout_map.setBlock_layout(regulate_point_3d1,0)
                layout_map.setBlock_layout(regulate_point_3d2,0)
                #------------------------------处理mp2-------------
                prev_mp2_block_1=layout_map.get_blocked_point(regulate_point_3d1,2) #获取mn0的source处原来被屏蔽的点
                prev_mp2_block_2=layout_map.get_blocked_point(regulate_point_3d2,2) #获取mn0的dest处原来被屏蔽的点
                prev_mp2_source.append(prev_mp2_block_1)
                prev_mp2_dest.append(prev_mp2_block_2)
                layout_map.setBlock_layout(regulate_point_3d1,2)
                layout_map.setBlock_layout(regulate_point_3d2,2)
                
                #-----------------------------------获取路径-----------------
                path_3d=AstarSearch_3d(layout_map,source,dest)
                len_path_3d=len(path_3d)
                if(len_path_3d==0):
                    len_path_3d=inf

                if((len_path_3d==inf or len_path_3d < min_port_reg_try )and m==min_port_reg_try+1):#如果直到最后也没有有效路径，则跳出
                    print("No valid path")
                    #-----------处理mp1--------------
                    layout_map.clrBlock_layout(regulate_point1,1)
                    layout_map.clrBlock_layout(regulate_point2,1)
                    layout_map.setBlock_layout(prev_mp1_block_1,1)
                    layout_map.setBlock_layout(prev_mp1_block_2,1)
                    #-----------处理mn0--------------
                    layout_map.clrBlock_layout(regulate_point_3d1,0)
                    layout_map.clrBlock_layout(regulate_point_3d2,0)
                    layout_map.setBlock_layout(prev_mn0_block_1,0)
                    layout_map.setBlock_layout(prev_mn0_block_2,0)
                    #-----------处理mp2--------------
                    layout_map.clrBlock_layout(regulate_point_3d1,2)
                    layout_map.clrBlock_layout(regulate_point_3d2,2)
                    layout_map.setBlock_layout(prev_mp2_block_1,2)
                    layout_map.setBlock_layout(prev_mp2_block_2,2)
                    #---------------处理端口-------------------
                    layout_map.setBlock_layout([[port_2[0],port_2[1]],[port_1[0],port_1[1]]],1)#mp1层封锁端口
                    continue
                    
                elif((len_path_3d==inf or len_path_3d < min_port_reg_try ) and m!=min_port_reg_try+1):#若不满足要求，且index不为2，则进行下一轮iteration
                    #-----------处理mp1--------------
                    layout_map.clrBlock_layout(regulate_point1,1)
                    layout_map.clrBlock_layout(regulate_point2,1)
                    layout_map.setBlock_layout(prev_mp1_block_1,1)
                    layout_map.setBlock_layout(prev_mp1_block_2,1)
                    #-----------处理mn0--------------
                    layout_map.clrBlock_layout(regulate_point_3d1,0)
                    layout_map.clrBlock_layout(regulate_point_3d2,0)
                    layout_map.setBlock_layout(prev_mn0_block_1,0)
                    layout_map.setBlock_layout(prev_mn0_block_2,0)
                    #-----------处理mp2--------------
                    layout_map.clrBlock_layout(regulate_point_3d1,2)
                    layout_map.clrBlock_layout(regulate_point_3d2,2)
                    layout_map.setBlock_layout(prev_mp2_block_1,2)
                    layout_map.setBlock_layout(prev_mp2_block_2,2)
                    print("failed, try again")
                    continue #跳到for m循环起始处，m-1后继续测试
                else:#找到有效路径  

                    #-----------处理mp1--------------
                    layout_map.clrBlock_layout(regulate_point1,1)
                    layout_map.clrBlock_layout(regulate_point2,1)
                    layout_map.setBlock_layout(prev_mp1_block_1,1)
                    layout_map.setBlock_layout(prev_mp1_block_2,1)
                    #-----------处理mn0--------------
                    layout_map.clrBlock_layout(regulate_point_3d1,0)
                    layout_map.clrBlock_layout(regulate_point_3d2,0)
                    layout_map.setBlock_layout(prev_mn0_block_1,0)
                    layout_map.setBlock_layout(prev_mn0_block_2,0)
                    #-----------处理mp2--------------
                    layout_map.clrBlock_layout(regulate_point_3d1,2)
                    layout_map.clrBlock_layout(regulate_point_3d2,2)
                    layout_map.setBlock_layout(prev_mp2_block_1,2)
                    layout_map.setBlock_layout(prev_mp2_block_2,2)
                    #---------------处理端口-------------------
                    layout_map.setBlock_layout([[port_2[0],port_2[1]],[port_1[0],port_1[1]]],1)#mp1层封锁端口
                    layout_map.setBlock(path_3d)
                    #print("succeeded")
                    break#跳出for m 循环 不继续执行

            temp=[coord_info[i][1],coord_info[i][0]]#temp来重新排列coord_info

            path_min_temp=min_path
            new_path=[]
            len_path=len(path_min_temp)
            if(len_path>=min_port_reg_try):#minimal path len acceptable is ?
                print("succeed")
                if(display==1):
                    print("--------------------show result,path:{0}-----------------------".format(i+1))
                showMap_all(map_list)

                for k in range(0,len_path):
                    new_path.append([path_min_temp[k][0]+map_origin[0],path_min_temp[k][1]+map_origin[1]])#把还原的path坐标写入new path中
                #print(new_path)
                index_seq=get_index_sequence(new_path,coord_info[i][1][1],last_check_index(coord_info[i][0][1]))#获取path的index序列
                #print(index_seq)
                rtype_seq=get_route_type(index_seq)#获取path的走线类型
                std_path_coord=[]
                for p in range(0,len_path-1):#转为pcell坐标
                    new_path[p]=[new_path[p][0]*layout_unit_len,new_path[p][1]*layout_unit_len]
                    std_path_coord.append(new_path[p])
                    new_path[p]=pcell_coord(new_path[p],index_seq[p])
                #设置最后一个点的pcell坐标
                new_path[-1]=[new_path[-1][0]*layout_unit_len,new_path[-1][1]*layout_unit_len]
                std_path_coord.append(new_path[-1])
                new_path[-1]=pcell_coord(new_path[-1],last_check_index(index_seq[-1]))
                #分析path及其index、走线类型，获取一个pycell程序可读的info list
                info=analyze_path(new_path,return_path,index_seq,rtype_seq,std_path_coord)
                print("source:{0} to destination:{1}".format(new_path[0],new_path[-1]))
                #将info list转变为pycell，生成script记入script_all
                script=path_to_pcell(info,new_path,index_seq)
                script_all.append(script)
            else:#若没有有效的path，则输出失败信息到logger
                print("No valid path from {0} to {1}".format(source,dest))
                failed_path.append(i)
                log_temp="Instance {0} port {1} ({2},{3}) to Instance {4} port {5} ({6},{7})".format(connection_info[i][2],connection_info[i][3].replace("wire",""),
coord_info[i][1][0][0],coord_info[i][1][0][1],connection_info[i][0],connection_info[i][1].replace("wire",""),coord_info[i][0][0][0],coord_info[i][0][0][1])
                log_failed_coord.append(log_temp)
        #记录本次布线循环的所有信息，包括利用率、script、失败的路径index、log信息等
        util=map_list[2].get_map_util()
        script_summary.append(script_all)
        failed_path_summary.append(failed_path)
        util_summary.append(util)
        log_summary.append(log_failed_coord)

    coord_summary=[]#同上面的summary
    connection_summary=[]#同上面的summary

    invalid_loop=False #retry loop检测，若重新排序后和之前某次仿真的序列一样，说明后续所有重排操作进入无限循环
    for t in range(0,try_num):    
        run_routing(0)#运行布线函数
        print("Map utilization:{0}%".format(util_summary[t][2]/100))
        curr_script_len=len(script_summary[t])#获取本次布线的script长度，来判断本次布线成功与否
        if(curr_script_len==coord_len):#如果成功则终止retry程序
            print("Succeed at cycle {0}, exit searching program".format(t))
            break
        #重新排布coord_info和connection_info，恢复所有map，准备下一次布线
        print("Rearranging connection info sequence and reference map...")
        coord_summary.append(coord_info)
        coord_temp=[]
        connection_summary.append(connection_info)
        connection_temp=[]
        for i in failed_path_summary[t]:
            coord_temp.append(coord_info[i])
            connection_temp.append(connection_info[i])
        for i in range(0,coord_len):
            if(i in failed_path_summary[t]):
                pass
            else:
                coord_temp.append(coord_info[i])
                connection_temp.append(connection_info[i])
        for i in range(0,coord_len):
            coord_info[i]=coord_temp[i]
            connection_info[i]=connection_temp[i]
            
        #检测是否出现重复的coord_info
        '''for i in range(0,t):
            if(coord_info==coord_summary[t]):
                invalid_loop=True
                break
        if(invalid_loop):#若出现重复coord_info则跳出retry程序
            print("Invalid retry loop, exit")
            break'''
        for j in range(0,map_for_search_path_num):
            for x in range(width):
                for y in range(height):
                    map_list[j].setValue([x,y],layout_map_reg.getValue([x,y]))
        if(t<try_num-1):#根据retry的次数来输出不同的信息
            print("Start retry program, cycle {0}".format(t+1))
        else:
            print("Reach the largest number of retry cycles, exit")
    #若超出retry的次数仍未break，则从所有布线结果中挑选最优解
    len_script=len(script_summary)
    script_len_list=[]
    for l in range(0,len_script):#计算所有布线结果的script长度
        script_len_list.append(len(script_summary[l]))
    max_script=max(script_len_list)#获取script长度最大值
    temp_max_index = [(i, script_len_list[i]) for i in range(len(script_len_list))]#查找最大值对应的index
    max_index_list=[i for i, n in temp_max_index if n == max_script]#查找最大值对应的index
    if(len(max_index_list)==1):#如果只有一个最大值则直接选这一个
        best_index=max_index_list[0]
    else:#如果有多个最大值则比较map利用率，选最小的 #如果多个map利用率相同则取第一个，如果以后有别的影响因素再进一步筛选
        util_temp_list=[]
        for u in max_index_list:
            util_temp_list.append(util_summary[u][2])
        util_min_index=util_temp_list.index(min(util_temp_list))
        best_index=max_index_list[util_min_index]
    #print(max_script_length_index)
    for s in range(0,script_len_list[best_index]):#输出最长script
        for script_str in script_summary[best_index][s]:
            print(script_str,file=text)


    print("-------------------------summary for block {0} to {1} ----------------------------".format(range_list[0],range_list[1]))#以下输出summary信息
    if(len_script==1):
        print("All path finished successfully without running retry program")
    else:
        print("Best result is from cycle {0}".format(best_index+1))
        if(len(log_summary[best_index])!=0):
            for failed_path in log_summary[best_index]:
                print("Failed:{0}".format(failed_path))
        else:
            print("All path finished successfully after running retry program")
    print("Current map utilization is {0}%".format(util_summary[best_index][2]/100))
    #print(util_summary[best_index])

    print("--------------------------------------------------------------------------------------------------")
    text.close()
    return [log_summary[best_index],util_summary[best_index]]
                break
        if(invalid_loop):#若出现重复coord_info则跳出retry程序
            print("Invalid retry loop, exit")
            break
        for j in range(0,map_for_search_path_num):
            for x in range(width):
                for y in range(height):
                    map_list[j].setValue([x,y],layout_map_reg.getValue([x,y]))
        for j in range(len(map_ptl_list)):
             for x in range(width):
                for y in range(height):
                    map_ptl_list[j].setValue([x,y],layout_map_port.getValue([x,y]))          
        if(t<try_num-1):#根据retry的次数来输出不同的信息
            print("Start retry program, cycle {0}".format(t+1))
        else:
            print("Reach the largest number of retry cycles, exit")
    #若超出retry的次数仍未break，则从所有布线结果中挑选最优解
    len_script=len(script_summary)
    script_len_list=[]
    for l in range(0,len_script):#计算所有布线结果的script长度
        script_len_list.append(len(script_summary[l]))
    max_script=max(script_len_list)#获取script长度最大值
    temp_max_index = [(i, script_len_list[i]) for i in range(len(script_len_list))]#查找最大值对应的index
    max_index_list=[i for i, n in temp_max_index if n == max_script]#查找最大值对应的index
    if(len(max_index_list)==1):#如果只有一个最大值则直接选这一个
        best_index=max_index_list[0]
    else:#如果有多个最大值则比较map利用率，选最小的 #如果多个map利用率相同则取第一个，如果以后有别的影响因素再进一步筛选
        util_temp_list=[]
        for u in max_index_list:
            util_temp_list.append(util_summary[u][2])
        util_min_index=util_temp_list.index(min(util_temp_list))
        best_index=max_index_list[util_min_index]
    #print(max_script_length_index)
    for s in range(0,script_len_list[best_index]):#输出最长script
        for script_str in script_summary[best_index][s]:
            print(script_str,file=text)


    print("-------------------------summary for block {0} to {1} ----------------------------".format(range_list[0],range_list[1]))#以下输出summary信息
    if(len_script==1):
        print("All path finished successfully without running retry program")
    else:
        print("Best result is from cycle {0}".format(best_index+1))
        if(len(log_summary[best_index])!=0):
            for failed_path in log_summary[best_index]:
                print("Failed:{0}".format(failed_path))
        else:
            print("All path finished successfully after running retry program")
    print("Current map utilization is {0}%".format(util_summary[best_index][2]/100))
    #print(util_summary[best_index])

    print("--------------------------------------------------------------------------------------------------")
    text.close()
    return [log_summary[best_index],util_summary[best_index]]


In [87]:
def regulate_route_3d(coord,index,num):
    abs_list=[]
    if(index==1):
        for i in range(0,num):
            for j in range(1,4):
                regulate_p1=[coord[0]-i,coord[1]+j]
                regulate_p2=[coord[0]-i,coord[1]-j]
                abs_list.append(regulate_p1)
                abs_list.append(regulate_p2)
            abs_list.append([coord[0]-i,coord[1]])
    elif(index==2):
        for i in range(0,num):
            for j in range(1,4):
                regulate_p1=[coord[0]+j,coord[1]-i]
                regulate_p2=[coord[0]-j,coord[1]-i]
                abs_list.append(regulate_p1)
                abs_list.append(regulate_p2)
            abs_list.append([coord[0],coord[1]-i])

    elif(index==3):
        for i in range(0,num):
            for j in range(1,4):
                regulate_p1=[coord[0]+i,coord[1]+j]
                regulate_p2=[coord[0]+i,coord[1]-j]
                abs_list.append(regulate_p1)
                abs_list.append(regulate_p2)
            abs_list.append([coord[0]+i,coord[1]])

    elif(index==4):
        for i in range(0,num):
            for j in range(1,4):
                regulate_p1=[coord[0]+j,coord[1]+i]
                regulate_p2=[coord[0]-j,coord[1]+i]
                abs_list.append(regulate_p1)
                abs_list.append(regulate_p2)
            abs_list.append([coord[0],coord[1]+i])

    return abs_list
def regulate_route_new(coord,index,num):
    abs_list=[]
    if(index==1):
        for i in range(0,num):
            for j in range(1,4):
                regulate_p1=[coord[0]-i,coord[1]+j]
                regulate_p2=[coord[0]-i,coord[1]-j]
                abs_list.append(regulate_p1)
                abs_list.append(regulate_p2)
    elif(index==2):
        for i in range(0,num):
            for j in range(1,4):
                regulate_p1=[coord[0]+j,coord[1]-i]
                regulate_p2=[coord[0]-j,coord[1]-i]
                abs_list.append(regulate_p1)
                abs_list.append(regulate_p2)
    elif(index==3):
        for i in range(0,num):
            for j in range(1,4):
                regulate_p1=[coord[0]+i,coord[1]+j]
                regulate_p2=[coord[0]+i,coord[1]-j]
                abs_list.append(regulate_p1)
                abs_list.append(regulate_p2)

    elif(index==4):
        for i in range(0,num):
            for j in range(1,4):
                regulate_p1=[coord[0]+j,coord[1]+i]
                regulate_p2=[coord[0]-j,coord[1]+i]
                abs_list.append(regulate_p1)
                abs_list.append(regulate_p2)
    return abs_list